<a href="https://colab.research.google.com/github/dviva1972/denvlaiva/blob/master/dll_hw_7_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## DLL

## Домашняя работа 7 | RNN - 3

## Иванов Денис

1. Возьмите англо-русскую пару фраз (https://www.manythings.org/anki/)
2. Обучите на них seq2seq по аналогии с занятием. Оцените полученное качество
3.  Попробуйте заменить GRU ячейки на lstm-ячейки
  Оцените качество во всех случаях

### 1. Импорт библиотек

In [13]:
from io import open
import unicodedata
import string
import re
import pandas as pd
import numpy as np
import random
from itertools import product
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
# Visualisation
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import sys
from google.colab import drive 

### 2. Импорт и предобработка текста

####2.1. 

In [ ]:
!wget https://www.manythings.org/anki/rus-eng.zip
!unzip rus-eng.zip

--2021-09-16 02:46:38--  https://www.manythings.org/anki/rus-eng.zip
Resolving www.manythings.org (www.manythings.org)... 104.21.92.44, 172.67.186.54, 2606:4700:3030::6815:5c2c, ...
Connecting to www.manythings.org (www.manythings.org)|104.21.92.44|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14385451 (14M) [application/zip]
Saving to: ‘rus-eng.zip’

rus-eng.zip         100%[===================>]  13.72M  --.-KB/s    in 0.1s    

2021-09-16 02:46:39 (134 MB/s) - ‘rus-eng.zip’ saved [14385451/14385451]

Archive:  rus-eng.zip
  inflating: rus.txt                 
  inflating: _about.txt              


In [ ]:
!head rus.txt

Go.	Марш!	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #1159202 (shanghainese)
Go.	Иди.	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #5898247 (marafon)
Go.	Идите.	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #5898250 (marafon)
Hi.	Здравствуйте.	CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #402127 (odexed)
Hi.	Привет!	CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #466968 (katjka)
Hi.	Хай.	CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #467233 (timsa)
Hi.	Здрасте.	CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #3803577 (marafon)
Hi.	Здоро́во!	CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #3854188 (marafon)
Hi.	Приветик!	CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #7234283 (marafon)
Run!	Беги!	CC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #1569978 (Biga)


In [ ]:
SOS_token = 0
EOS_token = 1
lang1 = 'rus'
lang2 = 'eng'

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [ ]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Zа-яА-Я.!?]+", r" ", s)
    return s

In [ ]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines    #data/%s-%s.txt
    # lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').read().strip().split('\n')
    lines = open('rus.txt', encoding='utf-8').read().strip().split('\n')
   
    # Split every line into pairs and normalize
    # pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    pairs = [[normalizeString(s) for s in l.split('\t')[:-1]] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang  = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

#### 2.__ Лирическое отступление от шаблона  

Структура предложений в модель по количеству слов

In [ ]:
rl = readLangs('eng', 'rus', reverse=False)[2]

Reading lines...


In [ ]:
rl[2000:2005]

[['be patient .', 'потерпите .'],
 ['be serious .', 'будь серьезным !'],
 ['be serious .', 'будьте серьезны !'],
 ['be serious .', 'будьте серьезным !'],
 ['be serious .', 'будьте серьезнои !']]

In [ ]:
df_temp = pd.DataFrame(rl, columns = ['eng', 'rus'])
df_temp['eng_words'] =df_temp['eng'].str.split().apply(len)
df_temp['rus_words'] =df_temp['rus'].str.split().apply(len)
df_temp['min_words'] =df_temp[['rus_words', 'eng_words']].min(axis=1)
df_temp.tail()

,eng,rus,eng_words,rus_words,min_words
431092,death is something that we re often discourage...,смерть это зачастую то разговоры или даже мысл...,41,34,34
431093,at a moment when our economy is growing our bu...,в тот момент когда наша экономика растет наши ...,43,36,36
431094,since there are usually multiple websites on a...,поскольку саитов посвященных какои либо теме к...,46,40,40
431095,it s so easy to write good example sentences t...,создавать хорошие предложения примеры так легк...,50,35,35
431096,doubtless there exists in this world precisely...,несомненно для каждого мужчины в этом мире где...,102,83,83


In [ ]:
df_temp['min_w_gr']  =pd.cut(df_temp['min_words'],[-9,4,5,6,7,8,9,10,20,9999])
df_temp['min_w_gr'].value_counts()

(4, 5]        93003
(5, 6]        90976
(-9, 4]       84070
(6, 7]        69871
(7, 8]        43514
(8, 9]        24212
(10, 20]      13044
(9, 10]       12300
(20, 9999]      107
Name: min_w_gr, dtype: int64

In [ ]:
eng_prefixes = ("i am ", "i m ",
                "he is", "he s ",
                "she is", "she s",
                "you are", "you re ",
                "we are", "we re ",
                "they are", "they re ")
 
df_temp['prefi'] = [df_temp.loc[i,'eng'].startswith(eng_prefixes) 
                                                    for i in df_temp.index]

df_temp['prefi'].value_counts()

False    400933
True      30164
Name: prefi, dtype: int64

In [ ]:
df_temp[df_temp['prefi']==True].min_words.value_counts()

4     6764
5     6213
6     5196
7     4032
3     3445
8     2311
9     1105
10     576
11     256
12     123
13      51
14      34
2       31
15      10
16      10
18       3
17       3
20       1
Name: min_words, dtype: int64

In [ ]:
df_temp[(df_temp['prefi']==True)&(df_temp['min_words']>3)&(df_temp['min_words']<20)].shape[0]

26687

In [ ]:
df_temp[(df_temp['min_words']>3)&(df_temp['min_words']<7)].shape[0]

247252

In [ ]:
df_temp[(df_temp['min_words']>3)&(df_temp['min_words']<20)].shape[0]

410142

Примеры, на которых обучение пойдет с трудом / или не получится вовсе:

(тоже надо будет что то делать)

In [ ]:
df_temp['delta_len'] = abs(df_temp['eng_words'] - df_temp['rus_words'])
df_temp['delta_len'].value_counts()

1     173425
2     103033
0      92604
3      41133
4      14368
5       4636
6       1358
7        374
8         94
9         33
10        18
11        12
12         5
13         2
19         1
15         1
Name: delta_len, dtype: int64

C личными долгами и женщинами тоже надо что то делать

In [ ]:
 df_temp[(df_temp['delta_len']>10)&(df_temp['min_words']<20)].iloc[:,[0,1,2,3,-1]][2:11]

,eng,rus,eng_words,rus_words,delta_len
430037,i promise i ll pay you back the money that i o...,обещаю что верну тебе долг до конца месяца .,20,9,11
430039,i promise i ll pay you back the money that i o...,обещаю что верну вам долг до конца месяца .,20,9,11
430041,i promise that i ll pay you back the money i o...,обещаю что верну тебе долг до конца месяца .,20,9,11
430043,i promise that i ll pay you back the money i o...,обещаю что верну вам долг до конца месяца .,20,9,11
430375,i promise that i ll pay you back the money tha...,обещаю что верну тебе долг до конца месяца .,21,9,12
430377,i promise that i ll pay you back the money tha...,обещаю что верну вам долг до конца месяца .,21,9,12
430736,it had the head of a woman the body of a lion ...,у него была голова женщины тело льва крылья пт...,24,13,11
430831,i ve heard it said that it s harder to please ...,я слышал что женщине труднее угодить чем мужчи...,26,15,11
430832,i ve heard it said that it s harder to please ...,я слышал что женщину труднее удовлетворить чем...,26,15,11


Для усиления обучения:
-  исключим из модели предложения состоящие из 1-2 слов (< 4 обьектов включая EOS) - ~ 20% выборки в которых нет контекста - нечему учиться rnn
- предложения длиной > 20 слов << 0.1% выборки
- (действительно разные) группировки объектов в работу: 

4 - 20 объектов (включая EOS  и 2 префикса) - при наличии префикса - 26 688 наблюдений

4 - 6 объектов (включая EOS) - 3-5 слов  - 247 252 наблюдений

4 - 20 объектов - 410 142 наблюдений


#### 2.2.

In [ ]:
def filterPair(p, prefi, max_):
    if prefi == None:
        return  len(p[0].split(' ')) >3 and \
                len(p[1].split(' ')) >3 and \
                len(p[0].split(' ')) < max_ and \
                len(p[1].split(' ')) < max_ 
    else:
        return  len(p[0].split(' ')) >3 and \
                len(p[1].split(' ')) >3 and \
                len(p[0].split(' ')) < max_ and \
                len(p[1].split(' ')) < max_ and \
                p[0].startswith(prefi) #

def filterPairs(pairs, pref, max_l):
    return [pair for pair in pairs if filterPair(pair, pref, max_l)]

In [ ]:
def prepareData(lang1, lang2, pre, max_le, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs, pre, max_le)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

In [ ]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensorsFromPair(pair, input_l, output_l):
    input_tensor = tensorFromSentence(input_l, pair[0])
    target_tensor = tensorFromSentence(output_l, pair[1])
    return (input_tensor, target_tensor)

### 3. Архитектура сети

#### 3.1. Encoder




In [ ]:
class EncoderRNN2(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers, rnn_type):
        super(EncoderRNN2, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.rnn = rnn_type(hidden_size, hidden_size, n_layers)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.rnn(output, hidden)
        return output, hidden

    def initHidden(self, n_layers):
        return torch.zeros(n_layers, 1, self.hidden_size, device=device)

#### 3.2. Decoder - basic

In [ ]:
class DecoderRNN2(nn.Module):
    def __init__(self, hidden_size, output_size, n_layers, rnn_type):
        super(DecoderRNN2, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.rnn       = rnn_type(hidden_size, hidden_size, n_layers)
        self.out       = nn.Linear(hidden_size, output_size)
        self.softmax   = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output         = self.embedding(input).view(1, 1, -1)
        output         = F.relu(output)
        output, hidden = self.rnn(output, hidden)
        output         = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self, n_layers):        
        return torch.zeros(n_layers, 1, self.hidden_size, device=device)

#### 3.3. Train + Teacher forcing

In [ ]:
def train(input_tensor, target_tensor, 
          encoder, decoder, encoder_optimizer, decoder_optimizer, 
          criterion, max_length, rnn_ty, n_layers, t_f_ratio):
    
    if rnn_ty.__name__ == 'LSTM':
        encoder_hidden = (encoder.initHidden(n_layers), encoder.initHidden(n_layers))
    else:
        encoder_hidden = encoder.initHidden(n_layers)

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
                                            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < t_f_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                                                decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                                                decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

#### 3.4. Train + iters

In [ ]:
def trainIters(pair_sentenses, encoder, decoder, learning_rate, n_iters, 
               MAX_LENGTH, rnn_typ, nu_layers, teach_f_ratio, input_lan, output_lan, 
               print_every=5000, plot_every=500):
    
    start               = time.time()
    print_loss_total_loc = 0  
    print_loss_avg = 0

    encoder_optimizer = optim.Adam(encoder.parameters())
    decoder_optimizer = optim.Adam(decoder.parameters())

    training_pairs = [tensorsFromPair(random.choice(pair_sentenses), 
                                      input_lan, output_lan)
                                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder, decoder, 
                     encoder_optimizer, decoder_optimizer, criterion, 
                     MAX_LENGTH, rnn_typ, nu_layers, teach_f_ratio)
        print_loss_total_loc += loss
        
        if iter % print_every == 0:
            print_loss_avg = print_loss_total_loc / print_every
            print_loss_total_loc = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

            
    return print_loss_avg

#### 3.5. Evaluate

In [ ]:
def evaluate(encoder, decoder, sentence, max_length, n_layers, 
             Rnn, inp_lang, out_lang):
    
    with torch.no_grad():
        input_tensor = tensorFromSentence(inp_lang, sentence)
        input_length = input_tensor.size()[0]
        
        if Rnn.__name__ == 'LSTM':
            encoder_hidden = (encoder.initHidden(n_layers), encoder.initHidden(n_layers))
        else:
            encoder_hidden = encoder.initHidden(n_layers)

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(out_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words

In [ ]:
def evaluateRandomly(pair_sen, encoder, decoder, MAX_LENGTH, 
                     _lauers, rnn_, in_lang, o_lang,  n=10):
    for i in range(n):
        pair = random.choice(pair_sen)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0], MAX_LENGTH, 
                                 _lauers, rnn_, in_lang, o_lang)
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

#### 3.6. Вспомогательные функции

In [ ]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [ ]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

### 4. Обучение 

In [7]:
eng_prefixes = ("i am ", "i m ",
                "he is", "he s ",
                "she is", "she s",
                "you are", "you re ",
                "we are", "we re ",
                "they are", "they re "
                )

rnn_types           = [nn.RNN, nn.LSTM, nn.GRU]
cols                = ['Max_length', 'RNN_Type', 'Hidden_layers', 'Loss']
PATH7               = '/content/drive/My Drive/DLL/else/df_7_'

In [ ]:
def test_model_hw_7(max_length_sent, num_hidden_lay_list, hidden_size = 256,
                    learn_rate = 0.006, prefix = None, 
                    teach_force_ratio = 0.5, 
                    num_iters = 50000, every = 1000, num_file = 998):
    
    input_lang, output_lang, pair_s = prepareData(lang1, lang2, prefix, max_length_sent)
    print(random.choice(pair_s))
    
    df = pd.DataFrame(columns= cols)

    for rnn_type, num_layers in product(rnn_types, num_hidden_lay_list):

        encoder2 = EncoderRNN2(input_lang.n_words, hidden_size, num_layers, rnn_type
                               ).to(device)
        decoder2 = DecoderRNN2(hidden_size, output_lang.n_words, num_layers, rnn_type
                               ).to(device)

        print('\nMax_length = ', max_length_sent)                       
        print(f'{rnn_type.__name__}, {num_layers} layers\n\ntraining')
        loss_= trainIters(pair_s, encoder2, decoder2, learn_rate, num_iters, 
                          max_length_sent, rnn_type, num_layers, teach_force_ratio, 
                          input_lang, output_lang, every)

        print('\nevaluate\n')
        evaluateRandomly(pair_s, encoder2, decoder2, max_length_sent, 
                         num_layers, rnn_type, input_lang, output_lang)
        
        string          = [max_length_sent, rnn_type.__name__, num_layers, loss_]
        df.loc[len(df)] = string
        file_name       = PATH7 + str(num_file) + '.csv'
        df.to_csv(file_name)

#### 4.1.  Модель 1 - Предложения по шаблону eng_prefixes  длиной до 20 символов

In [ ]:
test_model_hw_7(max_length_sent = 20, num_hidden_lay_list = [1,3], 
                learn_rate = 0.0001, teach_force_ratio = 0.5, 
                prefix = eng_prefixes, num_iters = 20000, every = 1000,
                num_file = 1)

Reading lines...
Read 431097 sentence pairs
Trimmed to 26677 sentence pairs
Counting words...
Counted words:
rus 4161
eng 9867
['we re a little late .', 'мы немного опоздали .']

Max_length =  20
RNN, 1 layers

training
0m 13s (- 4m 10s) (1000 5%) 4.7400
0m 24s (- 3m 42s) (2000 10%) 4.6535
0m 36s (- 3m 26s) (3000 15%) 4.6309
0m 48s (- 3m 14s) (4000 20%) 4.6223
1m 0s (- 3m 1s) (5000 25%) 4.5174
1m 11s (- 2m 47s) (6000 30%) 4.5338
1m 23s (- 2m 35s) (7000 35%) 4.5621
1m 36s (- 2m 24s) (8000 40%) 4.4404
1m 48s (- 2m 12s) (9000 45%) 4.4698
2m 0s (- 2m 0s) (10000 50%) 4.4978
2m 11s (- 1m 47s) (11000 55%) 4.5060
2m 23s (- 1m 35s) (12000 60%) 4.4335
2m 35s (- 1m 23s) (13000 65%) 4.4484
2m 46s (- 1m 11s) (14000 70%) 4.4085
2m 58s (- 0m 59s) (15000 75%) 4.3827
3m 10s (- 0m 47s) (16000 80%) 4.5654
3m 22s (- 0m 35s) (17000 85%) 4.5059
3m 34s (- 0m 23s) (18000 90%) 4.4123
3m 46s (- 0m 11s) (19000 95%) 4.4816
3m 58s (- 0m 0s) (20000 100%) 4.3997

evaluate

> i m used to this cold weather .
= я привы

In [14]:
pd.pivot_table(pd.read_csv(PATH7 + '1.csv').iloc[:,1:],
               index=["RNN_Type"],
               values=["Loss"],
               columns=["Hidden_layers"],
               aggfunc=[np.max])

amax          
                   Loss          
Hidden_layers         1         3
RNN_Type                         
GRU            2.678194  3.198849
LSTM           3.085175  3.587489
RNN            4.399714  4.696941

#### 4.2.  Модель 2 - Предложения длиной 3-7 слов / без привязки к шаблону

In [ ]:
test_model_hw_7(max_length_sent = 8, num_hidden_lay_list = [1,3], 
                learn_rate = 0.0006, teach_force_ratio = 0.4, 
                prefix = None, num_iters = 25000, every = 2500, num_file = 2)

Reading lines...
Read 431097 sentence pairs
Trimmed to 214880 sentence pairs
Counting words...
Counted words:
rus 12062
eng 35605
['that s real helpful .', 'это деиствительно полезно .']

Max_length =  8
RNN, 1 layers

training
0m 57s (- 8m 41s) (2500 10%) 5.6184
1m 54s (- 7m 39s) (5000 20%) 5.4496
2m 52s (- 6m 41s) (7500 30%) 5.3956
3m 49s (- 5m 43s) (10000 40%) 5.3197
4m 46s (- 4m 46s) (12500 50%) 5.3540
5m 43s (- 3m 48s) (15000 60%) 5.3609
6m 40s (- 2m 51s) (17500 70%) 5.3353
7m 37s (- 1m 54s) (20000 80%) 5.3808
8m 35s (- 0m 57s) (22500 90%) 5.4145
9m 32s (- 0m 0s) (25000 100%) 5.4488

evaluate

> go up those stairs .
= поднимитесь по тои лестнице .
< это не принадлежит . <EOS>

> has tom changed much ?
= том сильно изменился ?
< том не доказано ? <EOS>

> tom kept digging .
= том продолжал копать .
< том очень хорошо . <EOS>

> we tricked you .
= мы обманули вас .
< мы знаем в . . <EOS>

> you didn t have any choice .
= у вас не было выбора .
< ты не трата . <EOS>

> public opinion

In [16]:
pd.pivot_table(pd.read_csv(PATH7 + '2.csv').iloc[:,1:],
               index=["RNN_Type"],
               values=["Loss"],
               columns=["Hidden_layers"],
               aggfunc=[np.max])

amax          
                   Loss          
Hidden_layers         1         3
RNN_Type                         
GRU            3.739626  4.317024
LSTM           4.034300  4.447996
RNN            5.448834  5.803250

#### 4.3.  Модель 3 - Предложения длиной до 20 символов все / почти полная выборка

In [ ]:
test_model_hw_7(max_length_sent =20, num_hidden_lay_list = [1,3], 
                learn_rate = 0.0006, teach_force_ratio = 0.4, 
                prefix = None, num_iters = 25000, every = 2500, num_file =3)

Reading lines...
Read 431097 sentence pairs
Trimmed to 409735 sentence pairs
Counting words...
Counted words:
rus 15912
eng 52630
['can i eat this ?', 'могу я это съесть ?']

Max_length =  20
RNN, 1 layers

training
1m 23s (- 12m 34s) (2500 10%) 5.5438
2m 47s (- 11m 9s) (5000 20%) 5.5047
4m 11s (- 9m 46s) (7500 30%) 5.6093
5m 34s (- 8m 21s) (10000 40%) 5.5485
6m 58s (- 6m 58s) (12500 50%) 5.5276
8m 22s (- 5m 35s) (15000 60%) 5.5786
9m 47s (- 4m 11s) (17500 70%) 5.5641
11m 11s (- 2m 47s) (20000 80%) 5.5887
12m 36s (- 1m 24s) (22500 90%) 5.6029
13m 59s (- 0m 0s) (25000 100%) 5.6353

evaluate

> why did you show that to tom ?
= зачем ты показал это тому ?
< вы знаете или ? <EOS>

> when was the first time that you came here ?
= когда вы первыи раз сюда пришли ?
< вы знаете думаешь ? <EOS>

> we re on the same side .
= мы на однои стороне .
< том сказал что . . . <EOS>

> i m not even sure i want to help you .
= я даже не уверен что хочу тебе помогать .
< том сказал что . . . <EOS>

> i am

In [17]:
pd.pivot_table(pd.read_csv(PATH7 + '3.csv').iloc[:,1:],
               index=["RNN_Type"],
               values=["Loss"],
               columns=["Hidden_layers"],
               aggfunc=[np.max])

amax          
                   Loss          
Hidden_layers         1         3
RNN_Type                         
GRU            4.099708  4.637227
LSTM           4.545713  4.710827
RNN            5.635264  5.623448

### 5. Выводы

In [27]:
df1 =pd.concat([pd.read_csv(PATH7 + '1.csv'),
                pd.read_csv(PATH7 + '2.csv'),
                pd.read_csv(PATH7 + '3.csv')])
df1['model'] = [1]*6 + [2]*6 +[3]*6

pd.pivot_table(df1.iloc[:,1:] ,
               index=["RNN_Type"],
               values=["Loss"],
               columns=["Hidden_layers", "model"],
               aggfunc=[np.max])

amax                                                  
                   Loss                                                  
Hidden_layers         1                             3                    
model                 1         2         3         1         2         3
RNN_Type                                                                 
GRU            2.678194  3.739626  4.099708  3.198849  4.317024  4.637227
LSTM           3.085175  4.034300  4.545713  3.587489  4.447996  4.710827
RNN            4.399714  5.448834  5.635264  4.696941  5.803250  5.623448